In [1]:
import sys
sys.path.append('../external/tslib')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import csv
import os
import time

random.seed(42)
np.random.seed(42)

### Preprocessing

In [2]:
# 读入CSV
df = pd.read_csv('../data/request_timeseries_all.csv')

# 把 timestamp 转成 date
df['date'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

# 删除原来的 timestamp
df = df.drop(columns=['timestamp'])

# 把 date 列放到最前面
df = df[['date', 'requests']]

# roll up evry 10 data points
group_size = 10
grouped = df.groupby(df.index // group_size)

# 构建结果 DataFrame：保留起始时间，计算归一化请求数
result = grouped.agg({
    "date": "first",
    "requests": lambda x: x.sum() / 120
})

# remove last row having less than 10 data points
if len(df) % group_size != 0:
    result = result[:-1]

result.to_csv('../data/request_timeseries_aggregated.csv', index=False)

In [11]:
# 分割数据集，将数据集分为训练集和测试集，后30%作为测试集

df = pd.read_csv('../data/request_timeseries_all.csv')
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]
train_data.to_csv('../data/request_timeseries_train.csv', index=False)
test_data.to_csv('../data/request_timeseries_test.csv', index=False)

### Experiment

In [2]:
# 内部用前80%作为训练集
from lib.forecast.tslib_util import (
    parse_tslib_args,
    run_tslib_training,
)

In [3]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'TimesNet',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--d_model', '16',
        '--d_ff', '32',
        '--des', 'Exp',
        '--itr', '1',
        '--top_k', '5',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_TimesNet_custom_ftS_sl36_ll18_pl36_dm16_nh8_el2_dl1_df32_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 1.2879006
	speed: 0.1723s/iter; left time: 365.4117s
Epoch: 1 cost time: 30.714282751083374
Epoch: 1, Steps: 185 | Train Loss: 1.0686498 Test Loss: 0.6211281
Validation loss decreased (inf --> 0.621128).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2518632
	speed: 0.3282s/iter; left time: 635.3329s
Epoch: 2 cost time: 26.86336374282837
Epoch: 2, Steps: 185 | Train Loss: 0.4250385 Test Loss: 0.4046817
Validation loss decreased (0.621128 --> 0.404682).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.3063534
	speed: 0.2834s/iter; left time: 496.1957s
Epoch: 3 cost time: 24.

In [3]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Transformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--d_model', '16',
        '--d_ff', '32',
        '--des', 'Exp',
        '--top_k', '5',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Transformer_custom_ftS_sl36_ll18_pl36_dm16_nh8_el2_dl1_df32_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.9325890
	speed: 0.0716s/iter; left time: 151.9091s
Epoch: 1 cost time: 11.3273606300354
Epoch: 1, Steps: 185 | Train Loss: 0.9857151 Test Loss: 1.1125985
Validation loss decreased (inf --> 1.112599).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.9454160
	speed: 0.1088s/iter; left time: 210.6893s
Epoch: 2 cost time: 9.119707345962524
Epoch: 2, Steps: 185 | Train Loss: 0.7827771 Test Loss: 0.9035861
Validation loss decreased (1.112599 --> 0.903586).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.7481211
	speed: 0.1049s/iter; left time: 183.7597s
Epoch: 3 cost time: 9.

In [4]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'PatchTST',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--n_heads', '2',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_PatchTST_custom_ftS_sl36_ll18_pl36_dm512_nh2_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.4385456
	speed: 0.0382s/iter; left time: 80.9713s
Epoch: 1 cost time: 6.0795183181762695
Epoch: 1, Steps: 185 | Train Loss: 0.6120434 Test Loss: 0.3671558
Validation loss decreased (inf --> 0.367156).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.4639916
	speed: 0.0902s/iter; left time: 174.6906s
Epoch: 2 cost time: 8.688975095748901
Epoch: 2, Steps: 185 | Train Loss: 0.4741322 Test Loss: 0.3858618
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.5068045
	speed: 0.0780s/iter; left time: 136.6064s
Epoch: 3 cost time: 6.551538705825806
Epoch: 3, Steps: 1

In [5]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'DLinear',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_DLinear_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 1.2265337
	speed: 0.0197s/iter; left time: 41.7053s
Epoch: 1 cost time: 3.339123249053955
Epoch: 1, Steps: 185 | Train Loss: 1.4302526 Test Loss: 1.5233018
Validation loss decreased (inf --> 1.523302).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.8414343
	speed: 0.0360s/iter; left time: 69.7744s
Epoch: 2 cost time: 2.803664445877075
Epoch: 2, Steps: 185 | Train Loss: 0.9364815 Test Loss: 1.0602875
Validation loss decreased (1.523302 --> 1.060287).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.6594452
	speed: 0.0410s/iter; left time: 71.8173s
Epoch: 3 cost time: 3.266

In [6]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Informer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Informer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.2685606
	speed: 0.0908s/iter; left time: 192.6410s
Epoch: 1 cost time: 16.81754159927368
Epoch: 1, Steps: 185 | Train Loss: 0.3120998 Test Loss: 0.2506303
Validation loss decreased (inf --> 0.250630).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1616997
	speed: 0.1953s/iter; left time: 378.0249s
Epoch: 2 cost time: 16.36061120033264
Epoch: 2, Steps: 185 | Train Loss: 0.1846308 Test Loss: 0.2379891
Validation loss decreased (0.250630 --> 0.237989).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1690879
	speed: 0.1878s/iter; left time: 328.8788s
Epoch: 3 cost time: 1

In [7]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'FEDformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)

Using GPU
Use GPU: cuda:0
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
 fourier enhanced cross attention used!
modes_q=27, index_q=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
modes_kv=18, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
>>>>>>>start training : long_term_forecast_ts_36_36_FEDformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.3294568
	speed: 0.5261s/iter; left time: 1115.7587s
Epoch: 1 cost time: 99.0444586277008
Epoch: 1, Steps: 185 | Train Loss: 0.6579430 Test Loss: 0.

In [8]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Autoformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)

Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Autoformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.7080758
	speed: 0.1259s/iter; left time: 267.0674s
Epoch: 1 cost time: 22.986199617385864
Epoch: 1, Steps: 185 | Train Loss: 0.9188149 Test Loss: 0.7926010
Validation loss decreased (inf --> 0.792601).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2466152
	speed: 0.2718s/iter; left time: 526.2251s
Epoch: 2 cost time: 23.935683250427246
Epoch: 2, Steps: 185 | Train Loss: 0.3220102 Test Loss: 0.7799638
Validation loss decreased (0.792601 --> 0.779964).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.3066706
	speed: 0.2680s/iter; left time: 469.1909s
Epoch: 3 cost tim

In [9]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Crossformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--inverse',
        '--use_gpu', 'True'
    ]
)

run_tslib_training(args)

Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Crossformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
train 5919
Scaler mean: [3.69341263]
Scaler std: [2.05966675]
test 1462
	iters: 100, epoch: 1 | loss: 0.2533940
	speed: 0.1433s/iter; left time: 303.9424s
Epoch: 1 cost time: 25.802560567855835
Epoch: 1, Steps: 185 | Train Loss: 0.4166359 Test Loss: 0.2348586
Validation loss decreased (inf --> 0.234859).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2841390
	speed: 0.2976s/iter; left time: 576.0925s
Epoch: 2 cost time: 25.237106561660767
Epoch: 2, Steps: 185 | Train Loss: 0.2058320 Test Loss: 0.2497153
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1734549
	speed: 0.2767s/iter; left time: 484.5783s
Epoch: 3 cost time: 25.203734874725342
Epoch: 3, St

### Forecast

In [18]:
import lib.forecast.tslib_util as tslib_util


forecaster = tslib_util.TimeseriesForecaster()
forecaster.setTransformer(    
    transformer=tslib_util.TimeseriesTransformer(
        date_start=1742028000, date_scale=120, scale=True,
        scale_mean=3.69341263, scale_std=2.05966675
    )
)


Using cpu or mps
Use CPU


In [23]:
result = forecaster.forecast(
        enc_stamp = np.arange(0, 360, 10),
        enc_data = np.array([
            5.5,
            4.4,
            4.316666666666666,
            4.491666666666666,
            5.35,
            5.433333333333334,
            4.908333333333333,
            4.375,
            4.091666666666667,
            3.525,
            4.966666666666667,
            5.025,
            4.691666666666666,
            4.733333333333333,
            4.75,
            4.95,
            5.358333333333333,
            6.191666666666666,
            5.7,
            6.15,
            6.508333333333334,
            5.733333333333333,
            6.266666666666667,
            4.925,
            4.316666666666666,
            5.041666666666667,
            5.791666666666667,
            5.7,
            6.491666666666666,
            6.225,
            4.883333333333334,
            4.891666666666667,
            4.066666666666666,
            3.5833333333333335,
            3.533333333333333,
            3.5,
        ]),
    )

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.mean_ = np.array([3.69341263])
scaler.scale_ = np.array([2.05966675])


# 真实值
true_values = np.array([
    2.233333333333333,
    1.941666666666666,
    2.208333333333333,
    2.083333333333333,
    2.008333333333333,
    1.891666666666666,
    1.383333333333333,
    1.158333333333333,
    1.175,
    0.958333333333333,
    0.816666666666666,
    1.033333333333333,
    0.758333333333333,
    0.841666666666666,
    0.808333333333333,
    0.691666666666666,
    0.875,
    0.866666666666666,
    0.858333333333333,
    1.125,
    1.366666666666666,
    1.375,
    1.833333333333333,
    2.241666666666666,
    2.658333333333333,
    3.441666666666667,
    4.191666666666666,
    3.85,
    4.191666666666666,
    3.225,
    3.15,
    2.825,
    4.175,
    4.783333333333333,
    4.925,
    4.625,
])

# 预测值
predicted_values = result

# 计算 MSE
mse = mean_squared_error(true_values, predicted_values)
mse



0.2641409895714611